# Data Cleaning & Validation - Evolution Data

**Author:** Alan Meeson <alan.meeson@capgemini.com>

**Date:** 2023-02-06

This notebook captures assumptions about the data, and validation of those assumptions.
This can serve as a template for the Cleaning and Validation stage of the ETL process for the evolution data.

Key findings are:
- The rows for South Africa are corrupted.  The `location` entry is missing the closing ' " '
- There are some `perc_sequences` entries which are below 0 by -0.01.  Typically for variants 'other' and 'non_who'
- There are some `perc_sequences` entries which are off by 0.01 in addition to the ones noted above.
- There is some duplication in counts between the variants 'other' and 'non_who'; if we include both in the sum, the totals for a location/day don't add up correctly.

In [ ]:
import os
import re
import csv
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_dir = '../data'
evolution_filename = os.path.join(data_dir, 'raw', 'Data1_Covid Variants evolution.csv')
fixed_evolution_filename = os.path.join(data_dir, 'fixed', 'Data1_Covid Variants evolution.csv')
cleaned_evolution_filename = os.path.join(data_dir, 'cleaned', 'covid_variants_evolution.parquet')

## Explore the Evolution Data

In [ ]:
evolution_df = pd.read_csv(evolution_filename)

In [ ]:
evolution_df.head()

### Validate formats

#### Location
Assume that locations are purely letters with optional brackets and start with a captial letter.

In [ ]:
location_pattern = re.compile('^[A-Z][A-Za-z\(\) ]+$')
all(evolution_df['location'].str.fullmatch(location_pattern))

In [ ]:
evolution_df[~evolution_df['location'].str.fullmatch(location_pattern)]

##### Correct the row formatting issue

We correct this here, rather than with the other cleaning, as if we do not then it will throw off all those tests too

In [ ]:
expected_columns = ['location', 'date', 'variant', 'num_sequences', 'perc_sequences', 'num_sequences_total']
expected_num_columns = len(expected_columns)

# If the output path does not yet exist, create it
if not os.path.exists(os.path.dirname(fixed_evolution_filename)):
    os.makedirs(os.path.dirname(fixed_evolution_filename))

# Scan each line in the csv file for quote errors resulting in too few columns, fix if possible
with open(fixed_evolution_filename, 'w', newline='') as outfp:
    writer = csv.writer(outfp, quoting=csv.QUOTE_ALL)
    
    with open(evolution_filename, 'r', newline='') as fp:
        reader = csv.reader(fp)
        
        for line in reader:
            # File is known to have no commas in actual data, so if any are present this is an error.            
            # Particularly if the line has fewer than the expected number of rows            
            if (len(line) < expected_num_columns) and any([',' in element for element in line]):
                new_line = []
                
                for element in line:
                    new_line.extend(element.split(','))
        
                new_line = [element.removesuffix('"') for element in new_line]
                
                if len(new_line) == expected_num_columns:
                    writer.writerow(new_line)
                else:
                    print("Could not correct bad line: %s" % line)
            else:
                writer.writerow(line)

In [ ]:
# Read the corrected data, and check the fix
evolution_df = pd.read_csv(fixed_evolution_filename)

location_pattern = re.compile('^[A-Z][A-Za-z\(\) ]+$')
all(evolution_df['location'].str.fullmatch(location_pattern))

#### Date

Assume dates are always in the DD.MM.YYYY format

In [ ]:
date_pattern = re.compile('^[0-3][0-9]\.[01][0-9]\.[12][0-9]{3}$')
all(evolution_df['date'].str.fullmatch(date_pattern))

#### Variant

Assume that variants start with something other than a number.

In [ ]:
variant_pattern = re.compile('^[A-Za-z\.:_][A-Za-z0-9\.:_]+$')
all(evolution_df['variant'].str.fullmatch(variant_pattern))

In [ ]:
evolution_df['variant'].unique()

#### The numbers

##### Num_Sequences

Should be a number greater than 0

In [ ]:
all(~evolution_df['num_sequences'].isna() & (evolution_df['num_sequences'] >= 0))

##### Perc Sequences 
Should be a number between 0 & 100

In [ ]:
is_perc_sequences_valid = ~evolution_df['perc_sequences'].isna() & (evolution_df['perc_sequences'] >= 0) & (evolution_df['perc_sequences'] <= 100)
all(is_perc_sequences_valid)

In [ ]:
evolution_df[~is_perc_sequences_valid]

##### Num Sequences Total

Should be a number greater than zero

In [ ]:
is_num_sequences_total_valid = ~evolution_df['num_sequences_total'].isna() & (evolution_df['num_sequences_total'] >= 0)
all(is_num_sequences_total_valid)

##### num_sequences should be less than or equal to num_sequences total

In [ ]:
is_part_of_the_whole = evolution_df['num_sequences'] <= evolution_df['num_sequences_total']
all(is_part_of_the_whole)

##### perc_sequences should be approximately equal to 100 * (num_sequences / num_sequences_total)
However, several are out by 0.01; so it's probably worth re-calculating these.

In [ ]:
is_the_perc_right = abs(evolution_df['perc_sequences'] - ((evolution_df['num_sequences'] / evolution_df['num_sequences_total']) * 100)) < 0.01
all(is_the_perc_right)

In [ ]:
sum(~is_the_perc_right)

In [ ]:
evolution_df[~is_the_perc_right]

##### sum of num_sequences grouped by date and country should be equal to the num_sequences_total for that group.
Which they are, but only if we exclude either "other" or "non_who", as it seems those categories overlap somehow.

In [ ]:
sequence_sums = evolution_df[['location', 'date', 'num_sequences']].groupby(['location', 'date']).sum()
sequence_totals = evolution_df[['location', 'date', 'num_sequences_total']].groupby(['location', 'date']).first()
combined_totals = pd.concat([sequence_sums, sequence_totals], axis=1)
does_it_add_up = combined_totals['num_sequences'] == combined_totals['num_sequences_total']
all(does_it_add_up)

In [ ]:
sum(~does_it_add_up)

In [ ]:
sequence_sums = evolution_df.loc[~(evolution_df['variant'] == 'non_who'), ['location', 'date', 'num_sequences']].groupby(['location', 'date']).sum()
sequence_totals = evolution_df[['location', 'date', 'num_sequences_total']].groupby(['location', 'date']).first()
combined_totals = pd.concat([sequence_sums, sequence_totals], axis=1)
does_it_add_up = combined_totals['num_sequences'] == combined_totals['num_sequences_total']
all(does_it_add_up)

## Perform the data cleaning

### Load a fresh copy of the data set and parse dates

In [ ]:
custom_date_parser = lambda x: datetime.datetime.strptime(x, "%d.%m.%Y")
evolution_df = pd.read_csv(fixed_evolution_filename, parse_dates=['date'], date_parser=custom_date_parser)

### Recalculate the perc_sequences

In [ ]:
evolution_df['perc_sequences'] = 100 * evolution_df['num_sequences'] / evolution_df['num_sequences_total']

### Re-apply the basic validation checks to see that it is all correct

In [ ]:
# Location is a string which starts with a captial letter, and may contain
# letters, spaces, brackets, dashes and apostrophes only.
location_pattern = re.compile('^[A-Z][A-Za-z\(\)\'\- ]+$')
all(evolution_df['location'].str.fullmatch(location_pattern))

In [ ]:
# Variant is a string which starts with something other than a number, and may contain
# letters, ., :, _ and numbers only.
variant_pattern = re.compile('^[A-Za-z\.:_][A-Za-z0-9\.:_]+$')
all(evolution_df['variant'].str.fullmatch(variant_pattern))

In [ ]:
# num_sequences is a number greater than or equal to zero, and is not null
all(~evolution_df['num_sequences'].isna() & (evolution_df['num_sequences'] >= 0))

In [ ]:
# Perc_sequences is a floating point number in the range 0 <= x <= 100.
is_perc_sequences_valid = ~evolution_df['perc_sequences'].isna() & \
    (evolution_df['perc_sequences'] >= 0) & \
    (evolution_df['perc_sequences'] <= 100)
all(is_perc_sequences_valid)

In [ ]:
# Num_sequences_total is a number greater than or equal to zero and is not null
is_num_sequences_total_valid = ~evolution_df['num_sequences_total'].isna() & (evolution_df['num_sequences_total'] >= 0)
all(is_num_sequences_total_valid)

In [ ]:
# Num_sequences should always be less than or equal to num_sequences_total
is_part_of_the_whole = evolution_df['num_sequences'] <= evolution_df['num_sequences_total']
all(is_part_of_the_whole)

In [ ]:
# If we exclude the non_who row, the sum of num_sequences for a given location/date pair will
# add up to the num_sequences_total for that location/date pair
sequence_sums = evolution_df.loc[
    ~(evolution_df['variant'] == 'non_who'), 
    ['location', 'date', 'num_sequences']
].groupby(['location', 'date']).sum()

sequence_totals = evolution_df[['location', 'date', 'num_sequences_total']].groupby(['location', 'date']).first()

combined_totals = pd.concat([sequence_sums, sequence_totals], axis=1)
does_it_add_up = combined_totals['num_sequences'] == combined_totals['num_sequences_total']
all(does_it_add_up)

### Create a parquet file for the results

In [ ]:
# If the output path does not yet exist, create it
if not os.path.exists(os.path.dirname(cleaned_evolution_filename)):
    os.makedirs(os.path.dirname(cleaned_evolution_filename))
    
evolution_df.to_parquet(cleaned_evolution_filename)